In [1]:
import pandas as pd
import glob
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import shutil

path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(path)
os.getcwd()#os.listdir()

'C:\\Users\\Asus\\Documents\\GitHub\\ANN_Itau'

In [ ]:
df = pd.read_csv('Datos/raw/Transaccion_train.csv', index_col=0) #Hacer un acumulativo de montos o trANSSACCIONES
df.Periodo = df.Periodo.astype('object') #Lo pasamos a str
df.id = df.id.astype('object') #Lo pasamos a str
df.Signo = df.Signo.astype(str)
df['Fecha'] = pd.to_datetime(df.Fecha)

#-.-.-.-.
#Subsetear por periodo:
if (not os.path.exists('./Datos/raw/transaction_subset')):# Crear carpeta ./transaction_subset
    os.mkdir('./Datos/raw/transaction_subset')
else:# Eliminar anteriores para reemplazarlos:
    shutil.rmtree('./Datos/raw/transaction_subset')
    os.mkdir('./Datos/raw/transaction_subset')
data={}
size={}
for x in df.Periodo.unique():
    print(f"P_{x}")
    data[f"P_{x}"]=df.loc[(df.Periodo ==x)]
    #print(f"df_Periodo{x}",globals()[f"df_Periodo{x}"].describe())
    size[f"P_{x}"]=data[f"P_{x}"].shape()[0]
    data[f"P_{x}"].to_csv('Datos/raw/transaction_subset/'+f"P_{x}"+'.csv',index=False)

In [ ]:
data = {}# HAY QUE PROBAR EL CHUNK
periodos=[201901, 201902, 201903, 201904, 201905, 201906, 201907,201908, 201909, 201910, 201911, 201912,
          202001, 202002, 202003,202004, 202005, 202006, 202007]
porcent_datos=[]
for filename in periodos:
    data[f"P_{filename}"] = pd.read_csv('Datos/raw/transaction_subset/'+f"P_{filename}"+'.csv')

In [ ]:
# datasets=[data['P_201901']['Id_Producto'],data['P_201904']['Id_Producto']]
def histograma(elemento='Id_Producto'):
    datasets=[dataset[elemento] for dataset in data.values()]
    plt.rcParams.update({'font.size': 80})
    plt.figure(figsize=(150, 50), dpi=10)
    plt.hist(datasets, label=periodos)
    plt.title(f"Histograma {elemento} por periodo")
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
data['P_201901'].shape()

In [ ]:
for i in data['P_201901'].columns:
    print(f'\t {i}')
    display(histograma(i))

In [ ]:
def prepro(dataset):
    print( "\t Describe Montos")
    display(dataset['Monto'])
    print( "\t Describe Categoricas")
    objec = dataset.select_dtypes(include=['object'])
    display(objec.describe())

In [ ]:
primer_quan, cuar_quan = df_Periodo201912.Monto.quantile([.25, .75])

hist_monto=df.loc[(df_Periodo201912.Monto > primer_quan) & (df_Periodo201912.Monto < cuar_quan)]

In [ ]:
df_1 = df[df['Periodo']==201901]

In [ ]:
df_1.head()

In [ ]:
count = df_1.groupby('Id_Producto')['Id_Producto'].count()
print(count)

In [ ]:
count/count.sum()

In [ ]:
df_1.describe()